# Colaborative filtering

Colaborative filtering is a type of recommender systems that makes recommendations (predictions) based only on collected ratings/preferences of many users. To quote Wikipedia:

> In the newer, narrower sense, collaborative filtering is a method of making automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on an issue, A is more likely to have B's opinion on a different issue than that of a randomly chosen person. 

### Rating matrix

The user ratings can be collected in a rating matrix

$$R_{um}$$

each row of this matrix corresponds to one user and each column to one item (movie in this example). 
 An entry $R_{um}$ gives the rating of user $u$ for item $m$. This matrix has size $N_u\times N_m$ where $N_u$ is the number of users and $N_m$ is the numbers of items. This can be a very large number. However ss each of the users has usually rated a (very) small sample of items this matrix is sparse! In case of e.g. Netflix it has only about $1\%$ of filled entries.  

The aim of the recommender system is to predict the remaining $99\%$ of entries :) 

### Matrix Factorisation

One approach to this problem is to reduce the  rank of the rating matrix by representing it as a product of two matrices:

$$\hat{R}_{um} = U_{uk} M_{mk},\quad \hat{R}=U\cdot M^T$$

In this formulation each entry of the rating matrix $\hat R$ is a dot product of a row of users matrix $U$ and a row of movies matrix $M$. We can view each row of movie matrix as a set of _features_ and each row of the users matrix as _preferences_ for each of the features.

The number of features $K$ is a hyperparameter that we have to choose somehow. 

Reconstructing the rating matrix $\hat R$ amounts to estimating the matrices $U$ and $M$. This is alltogether  $N_u\times K + N_u\times K$ parameters. Depending on $K$ this is usually much smaller number then the number of all entries in $\hat R$.  

Estimation of $U$ and $M$ is done byminimazing the mean squared error:

$$J=\frac{1}{2}\sum_{<u,m>}\left(R_{um}-\hat{R}_{um}\right)^2$$

The sum is over a set of all non empty entries of matrix $R$ and we denote this by symbol $\langle u,m\rangle$.

Differentiating with respect to an element of the matrix $U$ we obtain:

$$\frac{\partial }{\partial {U_{uk}}} J = \sum_{<u,m>} \left(R_{um} - U_{uk'} M_{mk'}\right) M_{mk}$$

In the above formula idices $u$ and $k$ are fixed and the sum runs over all the movies $m$ rated by the user $u$. We also use the summation conventions according to which a duplicated index ($k'$ in this case) implies a summation over this index.

Equating this derivative to zero gives as an equation

$$\sum_{<u,m>} M_{mk}  M_{mk'} U_{uk'} = \sum_{<u,m>}  R_{um} M_{mk}$$

which can be rewritten in vector form:

$$ \left(\sum_{<u,m>}\vec{M}_{m}  \vec{M}_{m}^T\right) \vec{U}_{u} = \sum_{<u,m>}  R_{um} \vec{M}_{m}$$

here $\vec{U}_u$ is the $u$-th row of the matrix $U$ and similarly for $\vec{M}_m$. The summation is again over all movies $m$ rated by user $u$. 

In the same way we can derive another set of equation for the rows of the matrix $M$:

$$ \left(\sum_{<u,m>}\vec{U}_{u}  \vec{U}_{u}^T\right) \vec{M}_{m} = \sum_{<u,m>}  R_{um} \vec{U}_{u}$$

Minimazing the  $J$  requires solving both of those sets of equations simultaneuosly. This cannot be done in general and requires numerical methods. One of the is the iterative merthods of alterneting least squares.

### Alternating least squares

I if we consider $M$ fixed the equiations for the rows of $U$ are ordinary linear equations. Based on this observation the method consists of filling $M$ with same random starting values and the solving for $U$

$$  \vec{U}_{u} = \left(\sum_{<u,m>}\vec{M}_{m}  \vec{M}_{m}^T\right)^{-1}\sum_{<u,m>}  R_{um} \vec{M}_{m}$$

Then keeping $U$ fixed we solve for $M$

$$ \vec{M}_{m} =\left(\sum_{<u,m>}\vec{U}_{u}  \vec{U}_{u}^T\right)  \sum_{<u,m>}  R_{um} \vec{U}_{u}$$

and the whole process is reapeted until convergence. 

## Data 

In [1]:
import numpy as np
import numpy.linalg as la
import scipy as sp
from scipy import sparse
import pandas as pd


import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler

We will use the data from the Movielens. Please download and unpack the  ```ml-latest-small.zip``` from their [website](https://grouplens.org/datasets/movielens/latest/).

In [2]:
data = pd.read_csv('ml-latest-small/ratings.csv')

In [3]:
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
data.drop('timestamp', axis=1, inplace=True)

In [5]:
n_ratings = len(data)
data.shape

(100836, 3)

To find the number of users and movies in the data set we can use function ```nunique```:

In [6]:
n_users  = data['userId'].nunique()
n_movies = data['movieId'].nunique()
print(n_users, n_movies)

610 9724


### Problem 1

Find the size of the rating matrix, assuming $K=64$ calculate number of parameters we have to fit. 

In [7]:
n_of_parameters = (n_users + n_movies) * 64
print(n_of_parameters)

661376


In [8]:
sparse_rating_matrix = sp.sparse.csr_matrix((data.rating.values, (data.userId.values, data.movieId.values)))

In [9]:
sparse_rating_matrix.toarray()

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 4. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 2.5, 2. , ..., 0. , 0. , 0. ],
       [0. , 3. , 0. , ..., 0. , 0. , 0. ],
       [0. , 5. , 0. , ..., 0. , 0. , 0. ]])

Compare number of non empty entries in the rating matrix to number of parameters.

In [10]:
print("NON EMPTY ENTRIES: " + str(data.shape[0]))
print("num of parameters: " + str(n_of_parameters))

NON EMPTY ENTRIES: 100836
num of parameters: 661376


Compare number of paremeters to the size of available data. 

In [11]:
ratings_shape = sparse_rating_matrix.shape
print("Available data: " + str(ratings_shape[0] * ratings_shape[1]))
print("num of parameters: " + str(n_of_parameters))

Available data: 118295710
num of parameters: 661376


In [12]:
print('Number of movies that didnt appear in the dataset: ' + str(len(np.where(~sparse_rating_matrix.toarray().any(axis=0))[0]))) 

Number of movies that didnt appear in the dataset: 183886


#### Regularisation

Those calculations show that we still have more parameters to fit then the data. This can (and will) lead to serious overfitting. One way to mitigate that is to use regularisation of the parameters by constraining their size. 

This amounts to adding another term to the cost function $J$:

$$
J=\frac{1}{2}\sum_{<u,m>}\left(R_{um}-\hat{R}_{um}\right)^2+
\frac{\lambda}{2}\left(
||U||^2 +||M||^2
\right)
$$

where the norm of the matrix $||U||^2$ denotes the sum of all the squares of the matrix  entries

$$||U||^2=\sum_{u,k}U_{ik}^2$$

The $\lambda$ is another hyperparameter to be estimated.  

After taking the gradient we obtain new sets of equations

$$ \vec{U}_{u} = \left(\sum_{<u,m>}\vec{M}_{m}  
\vec{M}_{m}^T+\lambda I\right)^{-1}
\sum_{<u,m>} R_{um}\vec{M}_{m}$$

$$ \vec{M}_{m} =\left(\sum_{<u,m>}\vec{U}_{u}  \vec{U}_{u}^T+\lambda I\right)  \sum_{<u,m>}  R_{um} \vec{U}_{u}$$

that we solve in the same iterative way. 

Before proceeding to implementation we need to split our data into train, test and validation sets. 

In [13]:
train = data.sample(frac=0.9, replace=False)

In [14]:
train.head()

,userId,movieId,rating
4458,28,4855,2.5
95010,599,129397,2.0
63585,414,4029,5.0
68231,439,4022,4.5
64926,414,133771,4.5


In [15]:
test = data.drop(train.index)

In [16]:
validation = test.sample(frac=0.5, replace=False)

In [17]:
validation.head()

,userId,movieId,rating
99778,610,3945,3.0
9407,63,92494,5.0
55654,368,2122,2.0
92642,599,29,3.5
6558,45,783,3.0


In [18]:
test =test.drop(validation.index)

In [19]:
test.head()

,userId,movieId,rating
0,1,1,4.0
19,1,349,4.0
71,1,1206,5.0
153,1,2395,5.0
190,1,2948,5.0


In [20]:
n_train_users  = train.userId.nunique()
n_train_movies = train.movieId.nunique()

In [21]:
print(n_train_users, n_train_movies)

610 9377


In [22]:
n_test_users  = test.userId.nunique()
n_test_movies = test.movieId.nunique()

In [23]:
print(n_test_users, n_test_movies)

555 2512


In [24]:
n_validation_users=validation.userId.nunique()
n_validation_movies = validation.movieId.nunique()

In [25]:
print(n_validation_users, n_validation_movies)

556 2468


When implementing the ALS algorithm we will need to traverse the ratings users by user and for each user iterate over all movies that she has rated. To this end we will define an auxiliary data structur

In [26]:
by_user = train.groupby('userId').apply(lambda g: list(zip(g.movieId, g.rating)))

This is a ```pandas.Series``` indexed by the  ```userId``` that contains lists of pairs ```(movieId, rating)``` for each movie rated by this partuclar user.

Similarly we will need to traverse list of all movies and for each movie traverse list of users that have rated it, so we create another auxiliary structure. 

In [27]:
by_movie = train.groupby('movieId').apply(lambda g: list(zip(g.userId, g.rating)))

One last point to remember is that we cannot(!) use the ```userId``` to index  rows of matrix $U$! This is because the userId are not garantied to be consecutive, especially after spliting the data into three different sets. The correct way of finding the index into the U matrix for given userId is to find it's location in the series:

```ui = by_user.index.get_loc(uid)```

Same applies to movies indexes

```mi = by_movie.index.get_loc(mid)```

In [28]:
test.head()

,userId,movieId,rating
0,1,1,4.0
19,1,349,4.0
71,1,1206,5.0
153,1,2395,5.0
190,1,2948,5.0


### Problem 2 

2.1 Implement the ALS algorithm in python. 

In [29]:
#http://danielnee.com/2016/09/collaborative-filtering-using-alternating-least-squares/
class AlternatingLeastSquares:
    def __init__(
            self,
            ratings,
            n_features: int = 64,
            n_iter: int = 10,
            λ: float = 1.0,
            𝜀: float = 0.1
    ):
        self.ratings = ratings
        self.n_features = n_features
        self.n_iter = n_iter
        
        self.λ = λ
        self.𝜀 = 𝜀
    
        self.U = np.random.randn(ratings.shape[0], n_features)
        self.U_eye = np.eye(self.U.shape[0])
        self.M = np.random.randn(n_features, ratings.shape[1])
        self.M_eye = np.eye(self.M.shape[1])
        self.features_eye = np.eye(n_features)
        
    def mean_squared_error(y_pred, y_true):
        return np.mean((y_pred - y_true) ** 2)

    def fit(self):
        for i in range(self.n_iter):
            print('iteration %d of %d' % (i + 1, self.n_iter))
            for i, user_ratings in enumerate(self.ratings):
                self.U[i] = self._factorise(user_ratings, M, self.λ, self.features_eye)
            for i, movie_ratings in enumerate(self.ratings.T):
                self.M[i] = self._factorise(movie_ratings, U, self.λ, self.features_eye)
        
        mse = self.mean_squared_error(self.predict(), self.ratings)
        print("Mean Squared Error: " + str(mse))
            
    def predict(self):
        return self.U.dot(self.M.T)
            
    def _factorise(self, ratings, X, λ, eye):
        return np.linalg.inv(X.T.dot(X) + (λ * eye)).dot(X.T).dot(ratings)

In [30]:
model = AlternatingLeastSquares(sparse_rating_matrix)

In [31]:
model.fit()

iteration 1 of 10
iteration 2 of 10
iteration 3 of 10
iteration 4 of 10
iteration 5 of 10
iteration 6 of 10
iteration 7 of 10
iteration 8 of 10
iteration 9 of 10
iteration 10 of 10


ValueError: shapes (611,64) and (193610,64) not aligned: 64 (dim 1) != 193610 (dim 0)

2.2 Check its predictions on the train, test and validation set. 

One problem  is that the validation and test sets may contain users and/or movies that are not present in the training set. In this case the prediction is impossible. We can check this at the level of caclulating the prediction score or clean up those sets beforehand.  We will clean up the sets using an inner join on two dataframes.  

First we prepare two dataframes that contain only index which constists of unique users(movies) Id from the training sets. 

In [ ]:
by_user_idx = pd.DataFrame(index = by_user.index)
by_movie_idx = pd.DataFrame(index=by_movie.index)

In [ ]:
len(test)

We now use this frames in join. Inner join keeps only the rows with keys apearing in both dataframes. 

In [ ]:
test=test.join(by_user_idx, on='userId', how='inner')
test=test.join(by_movie_idx, on='movieId', how='inner')

In [ ]:
len(test)

In [ ]:
validation=validation.join(by_user_idx, on='userId', how='inner')
validation=validation.join(by_movie_idx, on='movieId', how='inner')

2.3 Does the result depend on number of features (K)? How?

2.4 Does the result depend on $\lambda$? How?

### Biases

We can extend our model by adding additional "biases" and mean:

$$\hat{R}_{um} = U_{uk} M_{mk} + b_u +c_m +\mu$$

where $\mu$ is just overall mean (on training set):

$$\mu = \frac{1}{\sum_{<u,m> } 1}\sum_{<u,m>}R_{um}$$

We can calculate gradient with respwect to biases

$$\frac{\partial }{\partial {b_{u}}} J = \sum_{<u,m>} \left(R_{um} - U_{uk'} M_{mk'}- b_u -c_m -\mu\right) \cdot 1 $$

$$\sum_{<u,m>} \left(R_{um} - U_{uk'} M_{mk'}- b_u -c_m -\mu\right)  $$

$$\sum_{<u,m>}  b_u =\sum_{<u,m>} \left(R_{um} - U_{uk'} M_{mk'} -c_m -\mu\right)  $$

and obtain te final equations:

$$ \vec{U}_{u} = \left(\sum_{<u,m>}\vec{M}_{m}  
\vec{M}_{m}^T+\lambda I\right)^{-1}
\sum_{<u,m>} \left( R_{um} - b_u -c_m -\mu\right)\vec{M}_{m}$$

$$ \vec{M}_{m} =\left(\sum_{<u,m>}\vec{U}_{u}  \vec{U}_{u}^T+\lambda I\right)  \sum_{<u,m>} \left( R_{um} - b_u -c_m -\mu\right) \vec{U}_{u}$$

$$ b_u =\frac{1}{\lambda+\sum_{<u,m>} 1}\sum_{<u,m>} \left(R_{um} - U_{uk'} M_{mk'} -c_m -\mu\right)  $$

$$ c_m =\frac{1}{\lambda+\sum_{<u,m>} 1}\sum_{<u,m>} \left(R_{um} - U_{uk'} M_{mk'} -b_u -\mu\right)  $$

We solve this equations in the same way, we start from random values and  solve each equation by keeping all other varaibles fix. 

### Problem 3

3.1 Implement this version of the ALS algorithm and compare it's performance with the previous one.

3.2 Using the ```movies.csv``` set associate the movies with biases. Print top ten movies with highest bias, and top ten with lowest (negative). 